# 深層学習

教師あり機械学習の基礎となる回帰について学びます。
ボストンハウジングという有名問題

![deep1-fs8.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/57754/ebc23dd0-544b-7b72-185e-510a584aae8b.png)

![deep2-fs8.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/57754/49663032-edde-3f61-c00b-cc1dcf5d0704.png)



In [ ]:
## MNIST

MNIST は、「0」~「9」の手書き数字の画像データセットです。Yann LeCun らによって、手書き 文字データセットの NIST データベースから抽出&加工して、機械学習のチュートリアル用に作成 されて公開されています。

ここは、画像データ(手書きの文字)を説明変数 (X)、ラベルを目的変数 (y) として学習モデルを作成してみます。

画像データ (x_train, x_test) は、サイズ 28x28, 0~255 グレースケールの形式になっています。

# MNISTデータセット
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')
mnist
#(X_train, y_train), (X_test, y_test) = mnist.load_data()


## 今回のお題

プログラミングでは、複数の値をひとまとめのデータとして扱いたいケースが多くあります。

__もしデータをひとつひとつ変数で識別すると__:

```python
a1 = 1
a2 = 2
a3 = 3
print(a1, a2, a3)  #３つが限界だな..
```



## 演習問題

<div class="alert alert-info">

Let's try

`2 ** (1//2)` が、正しく $\sqrt{2}$ にならない理由を考えてみよう

</div>

<div class="alert alert-warning">

(プログラミングの)$N$個数える

プログラミングでは、原則、「**0から**$N-1$まで」のように数えます。

</div>

### 平均点

<div class="admonition tip">

**例題（平均点）**

期末試験は5人受験した。
点数が40点未満の生徒は全員，補習を受け，成績が40点になった。
5人の平均点を求めよ。

入力例：
```
10
65
100
30
95
```

出力例：
```
68
```

[AtCoder (JOI2014 予選)](https://atcoder.jp/contests/joi2014yo/tasks/joi2014yo_a)

</div>

__(解法) リストを使う場合__

1. 期末試験を記録する空の得点リスト `scores` を用意する
2. 5人分繰り返し、点数を読んで、`scores` に追加する 
3. 平均点は `sum(scores) // 5`
